In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import os 
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import historical_prerpo
from historical_prerpo import prepro

path = "../../data/raw/historical_data/*.csv"
file_lst = []
for fname in glob.glob(path):
    file_lst.append(fname)


The preprocessing function will loop each raw file and they will be saved as curated data respectively.

In [2]:
for file in file_lst:
    prepro(file, 'historical')

../../scripts/price.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['weekly_rent'] = df2['weekly_rent'].str.extract('(\d+)').astype(int)
../../scripts/price.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['weekly_rent'] = df2['weekly_rent'].str.extract('(\d+)').astype(int)
../../scripts/price.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

After looping and preprocess through all historical data, we concatenate them into one dataframe and save them seperately base on year. 

In [22]:
curated_path = "../../data/curated/historical/*.csv"
file_lst2 = []
for fname in glob.glob(curated_path):
    file_lst2.append(fname)

# combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in file_lst2 ])
combined_csv = combined_csv.drop(columns = "Unnamed: 0")
# run the following code if you want to generate a whole historical dataset. around 57MB. Thus, will not keep that in git for now.
# export to csv
# combined_csv.to_csv( "../../data/curated/historical/combined_historical.csv", index=False, encoding='utf-8-sig') 

In [73]:
print(f'{len(combined_csv)-len(combined_csv.drop_duplicates())} duplicates will be dropped')

89829 duplicates will be dropped


In [74]:
combined_csv = combined_csv.drop_duplicates()


In [76]:
len(combined_csv)

457359

In [77]:
#combined_csv['year'] = float(combined_csv['historical_dates'].dt.year)
combined_csv['year'] = pd.to_datetime(combined_csv.historical_dates).dt.year
combined_csv['month'] = pd.to_datetime(combined_csv.historical_dates).dt.month
combined_csv

<ipython-input-77-4f9d5aa4de0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv['year'] = pd.to_datetime(combined_csv.historical_dates).dt.year
<ipython-input-77-4f9d5aa4de0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv['month'] = pd.to_datetime(combined_csv.historical_dates).dt.month


,address,latitude,longitude,nbed,nbath,ncar,price,type,historical_dates,postcode,suburb,weekly_rent,year,month
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2,1,3,$380_per_week,House,2022-03-01,3451,CHEWTON,380,2022,3
2,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2,1,3,$380_per_week,House,2022-02-01,3451,CHEWTON,380,2022,2
3,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2,1,3,$380,House,2022-02-01,3451,CHEWTON,380,2022,2
4,"108 ADELAIDE STREET, CHEWTON",-37.093186,144.254132,2,1,2,$340_per_week,Residential Lease,2021-11-01,3451,CHEWTON,340,2021,11
6,"108 ADELAIDE STREET, CHEWTON",-37.093186,144.254132,2,1,2,$330_pw,Residential Lease,2020-06-01,3451,CHEWTON,330,2020,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677,"90 SHEPPARTON ZEERUST ROAD, ZEERUST",-36.31942,145.403559,2,1,1,$200_pw,House,2016-11-01,3634,ZEERUST,200,2016,11
5679,"90 SHEPPARTON ZEERUST ROAD, ZEERUST",-36.31942,145.403559,2,1,1,$200_pw,House,2016-01-01,3634,ZEERUST,200,2016,1
5680,"1/1B KINKORA ROAD, ZEERUST",-36.268493,145.386377,2,2,1,$5000_pw,Unit/apmt,2016-03-01,3634,ZEERUST,5000,2016,3
5682,"2/1 TAYLOR STREET, ZEERUST",-36.268493,145.386377,1,1,1,$320_pw,Unit/apmt,2015-11-01,3634,ZEERUST,320,2015,11


In [79]:
year_lst = list(set(combined_csv['year']))
for i in range(len(year_lst)):
    df = combined_csv[combined_csv.year == year_lst[i]]
    df.to_csv(f'../../data/curated/historical/{year_lst[i]}_historical_clean.csv')

In [ ]:
for file in file_lst2:
    os.remove(file)

In [69]:
# os.remove("../../data/curated/historical/combined_historical.csv")